In [ ]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, datasets
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import classification_report,accuracy_score
import pandas as pd
from sklearn.datasets import load_iris ##
import os
from sklearn.metrics import classification_report,accuracy_score
from sklearn import svm
from tqdm._tqdm_notebook import tqdm_notebook

In [ ]:
# Import landmarks
import lab2_landmarks as l2

def get_data():
    # Target Index from lab2_landmarks -> 2: Gender_label, 3: Emotion_label
    return l2.extract_features_labels(3)
    
x, y = get_data()

In [ ]:
# Train-test split
x_tr, x_te, y_tr, y_te = train_test_split(x, yT, train_size = 0.7)

In [ ]:
# sklearn functions implementation

def img_SVM(model, training_images, training_labels, test_images, test_labels):
    classifier = model
#     classifier = svm.SVC(kernel='linear')
    classifier.fit(training_images, training_labels)

    pred = classifier.predict(test_images)

#     print(pred)
#     print("Accuracy:", accuracy_score(test_labels, pred))

    return (pred, accuracy_score(test_labels, pred))


def reshapeX(x):
    return x.reshape((x.shape[0], x.shape[1] * x.shape[2]))

def reshapeY(y):
    return list(zip(*y))[0]

x_tr = reshapeX(x_tr)
x_te = reshapeX(x_te)
y_tr = reshapeY(y_tr)
y_te = reshapeY(y_te)

In [ ]:
# Model instantiation
models = []

for c in [1, 10, 100]:
    for degree in np.arange(2) + 2:
        for gamma in ['scale', 'auto']:
             for kernel in ['linear', 'rbf', 'poly']:
                    models.append(svm.SVC(kernel = kernel, C = c, degree = degree, gamma = gamma))

In [ ]:
# Run predictions
results = []

for model in tqdm_notebook(models):
    results.append((model, img_SVM(model, x_tr, y_tr, x_te, y_te)))